In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  


import atac_utils

import importlib
importlib.reload(atac_utils)

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
# f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
adata.obs.index

### cellname = f"{barcode}-1-{sample}-2023 Multiome-{condition_code}-0"

In [ ]:
cellnames = np.array(list(adata.obs.index.str.split('-')))
cellnames

In [ ]:
np.unique(cellnames[:,1])

In [ ]:
np.unique(cellnames[:,2])

In [ ]:
np.unique(cellnames[:,3])

In [ ]:
np.unique(cellnames[:,4])

In [ ]:
np.unique(cellnames[:,5])

In [ ]:
natsorted(np.unique(['-'.join(cellname) for cellname in cellnames[:,[2,4]]]))

# correct name

In [ ]:
organized_sample_dict = atac_utils.get_organized_sample_dict()
organized_sample_dict

In [ ]:
%%time
outdir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'

for cond_code, samples in organized_sample_dict.items(): 
    # if cond_code != 9: 
    #     continue
    
    cond_name = atac_utils.decode_cond(cond_code)
    print(cond_code, cond_name, samples)
    
    f_output = outdir + f'/pmat_l23scenic_v2_{cond_name}.h5ad'
    if os.path.isfile(f_output):
        print(f"skip {cond_name}")
        continue

    adatas_pk = []
    for sample in samples:
        print(sample)
        # rna L2/3 only
        adatasub = adata[adata.obs['sample']==sample]
        cell_names_rna = adatasub.obs.index.values

        # atac all cells
        f = f'/u/home/f/f7xiesnm/v1_multiome/atac_fragments/pmat_snap/pmat_l23scenic_v2_{sample}.h5ad'
        adata_pk = sc.read(f)

        cell_names_raw = adata_pk.obs.index.values
        cell_names_new = np.char.add(cell_names_raw.astype(str), f'-{sample}-2023 Multiome-{cond_code}-0')
        adata_pk.obs.index = cell_names_new

        # check compatibility
        cell_names_both = np.intersect1d(cell_names_new, cell_names_rna)
        print(len(cell_names_both)/len(cell_names_rna))

        adatas_pk.append(adata_pk[cell_names_both])
        
    adata_concat = sc.concat(adatas_pk)
    print(adata_concat)
    print(f_output)
    sc.write(f_output, adata_concat)